In [1]:
import rdkit
print(rdkit.__version__)
from rdkit import Chem
import numpy as np
import pandas as pd
from rdkit.Chem import AllChem, PandasTools

2021.03.2


<frozen importlib._bootstrap>:219: RuntimeWarning: to-Python converter for boost::shared_ptr<RDKit::FilterCatalogEntry const> already registered; second conversion method ignored.


In [2]:
'''
The activity data of compounds tested on CYP19A1 (homo sapiens) were downloaded from ChEMBL (CHEMBL1987). 
Only the compounds with IC50 tested and pChEMBL values calculated were selected for further processing and analysis. 
'''
dfic50 = pd.read_csv('./datasets/1_raw/Chembl_IC50.csv', delimiter=';')

In [3]:
len(dfic50)

2926

In [4]:
dfic50.dropna(subset = ['Smiles'],inplace=True)
len(dfic50)

2926

In [5]:
dfic50["Standard Units"].unique(), dfic50['Standard Relation'].unique()

(array(['nM', nan, 'ug.mL-1', 'mg kg-1'], dtype=object),
 array(["'='", "'>'", nan, "'>='"], dtype=object))

In [6]:
dfic50_1 = dfic50[dfic50['Standard Relation']=="'='"]
dfic50_2 = dfic50.loc[(dfic50['Standard Relation']=="'>'")|(dfic50['Standard Relation']=="'>='")]
dfic50_3 = dfic50[dfic50['Standard Relation'].isna()]
len(dfic50_1), len(dfic50_2), len(dfic50_3)


(2569, 281, 76)

In [7]:
dfic50_1.dropna(subset = ['pChEMBL Value'],inplace=True)
len(dfic50_1)

/tmp/ipykernel_3406031/1455268226.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfic50_1.dropna(subset = ['pChEMBL Value'],inplace=True)


2484

In [8]:
dfic50_1

,Molecule ChEMBL ID,Molecule Name,Molecule Max Phase,Molecular Weight,#RO5 Violations,AlogP,Compound Key,Smiles,Standard Type,Standard Relation,...,Target Name,Target Organism,Target Type,Document ChEMBL ID,Source ID,Source Description,Document Journal,Document Year,Cell ChEMBL ID,Properties
0,CHEMBL174579,NaN,NaN,275.15,0.0,3.66,9b,Brc1ccc2c(c1)CC/C2=C/c1c[nH]cn1,IC50,'=',...,Cytochrome P450 19A1,Homo sapiens,SINGLE PROTEIN,CHEMBL1144350,1,Scientific Literature,J Med Chem,2005.0,NaN,NaN
1,CHEMBL1825024,NaN,NaN,235.29,0.0,2.92,28,c1ccc(C(c2ccccc2)n2cncn2)cc1,IC50,'=',...,Cytochrome P450 19A1,Homo sapiens,SINGLE PROTEIN,CHEMBL1821589,1,Scientific Literature,Eur J Med Chem,2011.0,NaN,NaN
2,CHEMBL28,APIGENIN,NaN,270.24,0.0,2.58,Apigenin,O=c1cc(-c2ccc(O)cc2)oc2cc(O)cc(O)c12,IC50,'=',...,Cytochrome P450 19A1,Homo sapiens,SINGLE PROTEIN,CHEMBL1157768,1,Scientific Literature,Bioorg Med Chem Lett,2010.0,NaN,NaN
3,CHEMBL196328,NaN,NaN,314.18,0.0,4.67,BDBM8911,COc1ccc2cc(-c3cccnc3)ccc2c1Br,IC50,'=',...,Cytochrome P450 19A1,Homo sapiens,SINGLE PROTEIN,CHEMBL3886626,37,BindingDB Database,NaN,2016.0,NaN,NaN
5,CHEMBL306022,NaN,NaN,302.50,1.0,5.34,15b,CC[C@@H]1CC2C(CCC3(C)C(O)CCC23)C2(C)CCCC=C12,IC50,'=',...,Cytochrome P450 19A1,Homo sapiens,SINGLE PROTEIN,CHEMBL1129695,1,Scientific Literature,J Med Chem,1996.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2921,CHEMBL206351,NaN,NaN,237.30,0.0,3.58,BDBM8891,COc1ccc2c(c1)CCC(c1cccnc1)=C2,IC50,'=',...,Cytochrome P450 19A1,Homo sapiens,SINGLE PROTEIN,CHEMBL3886626,37,BindingDB Database,NaN,2016.0,NaN,NaN
2922,CHEMBL4632445,NaN,NaN,358.40,0.0,3.98,3a,CCC#CCOc1ccc2c(=O)c3ccc(Cn4ccnc4)cc3oc2c1,IC50,'=',...,Cytochrome P450 19A1,Homo sapiens,SINGLE PROTEIN,CHEMBL4610109,1,Scientific Literature,ACS Med Chem Lett,2020.0,NaN,NaN
2923,CHEMBL4634777,NaN,NaN,358.40,0.0,3.98,1a,CCC#CCOc1cc(Cn2ccnc2)c2oc3ccccc3c(=O)c2c1,IC50,'=',...,Cytochrome P450 19A1,Homo sapiens,SINGLE PROTEIN,CHEMBL4610109,1,Scientific Literature,ACS Med Chem Lett,2020.0,NaN,NaN
2924,CHEMBL4755831,NaN,NaN,383.45,0.0,4.54,26,Nc1ccc(C(Cn2ccnc2)=C(c2ccc(O)cc2)c2ccc(O)cc2)cc1,IC50,'=',...,Cytochrome P450 19A1,Homo sapiens,SINGLE PROTEIN,CHEMBL4715691,1,Scientific Literature,J Med Chem,2016.0,NaN,TIME = 0.5 hr


In [9]:
len(dfic50_1[dfic50_1['pChEMBL Value']>=6]),len(dfic50_1[dfic50_1['pChEMBL Value']<6])

(1361, 1123)

In [10]:
import sys
sys.path.append('/home/sliu/cyp_related_coding/CYP19_inhibitor_screening/')
'''
The following function used for molecule preprocessing is from 'RingSySTEMS' project: https://github.com/anya-chen/RingSystems/tree/master/Preprocessing/src
'''
from src import MoleculePreprocessor
from src.MoleculePreprocessor import MoleculePreprocessor
from src.MoleculePreprocessorExtended import MoleculePreprocessorExtended

In [11]:
moleculesProcessed = MoleculePreprocessorExtended.init_with_smiles(list(dfic50_1.Smiles))

In [12]:
from rdkit import RDLogger
moleculesProcessed.csp_wash()

RDKit WARNING: [19:12:57] Tautomer enumeration stopped at 347 tautomers: max transforms reached
[19:12:57] Tautomer enumeration stopped at 347 tautomers: max transforms reached
[19:12:58] Tautomer enumeration stopped at 473 tautomers: max transforms reached
RDKit WARNING: [19:12:58] Tautomer enumeration stopped at 473 tautomers: max transforms reached
RDKit WARNING: [19:12:58] Tautomer enumeration stopped at 390 tautomers: max transforms reached
[19:12:58] Tautomer enumeration stopped at 390 tautomers: max transforms reached
[19:12:59] Tautomer enumeration stopped at 330 tautomers: max transforms reached
RDKit WARNING: [19:12:59] Tautomer enumeration stopped at 330 tautomers: max transforms reached
RDKit WARNING: [19:12:59] Tautomer enumeration stopped at 290 tautomers: max transforms reached
[19:12:59] Tautomer enumeration stopped at 290 tautomers: max transforms reached
RDKit WARNING: [19:13:00] Tautomer enumeration stopped at 404 tautomers: max transforms reached
[19:13:00] Tautomer

In [13]:
preprocessedSmilesDict = moleculesProcessed.get_rawsmiles_smiles_dict()

In [14]:
preprocessedSmiles_df = pd.DataFrame(list(preprocessedSmilesDict.items()), columns=['rawSmiles','preprocessedSmiles'])

In [15]:
preprocessed_df = pd.merge(dfic50_1, preprocessedSmiles_df, 
                           left_on='Smiles', right_on='rawSmiles')

In [16]:
preprocessed_df.dropna(subset=['preprocessedSmiles'],inplace=True) 

In [17]:
preprocessed_df = preprocessed_df[preprocessed_df['preprocessedSmiles']!='']

In [18]:
len(preprocessed_df)

1997

In [19]:
# Use the loc method to set the values in the new column based on the values in the existing column
preprocessed_df.loc[preprocessed_df['pChEMBL Value']>=6, 'activity_classification'] = 'active'
preprocessed_df.loc[preprocessed_df['pChEMBL Value']<6, 'activity_classification'] = 'inactive'

In [20]:
def remove_stereochemistry(smiles):
    # Convert the SMILES string to an RDKit molecule object
    mol = Chem.MolFromSmiles(smiles)
    
    # Use the Chem.RemoveStereochemistry function to remove stereo informationz
    Chem.RemoveStereochemistry(mol)
    
    # Convert the molecule back to a SMILES string without stereochemistry
    return Chem.MolToSmiles(mol)

preprocessed_df['preprocessedSmiles'] = preprocessed_df['preprocessedSmiles'].apply(remove_stereochemistry)
preprocessed_df

,Molecule ChEMBL ID,Molecule Name,Molecule Max Phase,Molecular Weight,#RO5 Violations,AlogP,Compound Key,Smiles,Standard Type,Standard Relation,...,Document ChEMBL ID,Source ID,Source Description,Document Journal,Document Year,Cell ChEMBL ID,Properties,rawSmiles,preprocessedSmiles,activity_classification
0,CHEMBL174579,NaN,NaN,275.15,0.0,3.66,9b,Brc1ccc2c(c1)CC/C2=C/c1c[nH]cn1,IC50,'=',...,CHEMBL1144350,1,Scientific Literature,J Med Chem,2005.0,NaN,NaN,Brc1ccc2c(c1)CC/C2=C/c1c[nH]cn1,Brc1ccc2c(c1)CCC2=Cc1c[nH]cn1,active
2,CHEMBL28,APIGENIN,NaN,270.24,0.0,2.58,Apigenin,O=c1cc(-c2ccc(O)cc2)oc2cc(O)cc(O)c12,IC50,'=',...,CHEMBL1157768,1,Scientific Literature,Bioorg Med Chem Lett,2010.0,NaN,NaN,O=c1cc(-c2ccc(O)cc2)oc2cc(O)cc(O)c12,O=c1cc(-c2ccc(O)cc2)oc2cc(O)cc(O)c12,active
3,CHEMBL28,APIGENIN,NaN,270.24,0.0,2.58,35,O=c1cc(-c2ccc(O)cc2)oc2cc(O)cc(O)c12,IC50,'=',...,CHEMBL1156709,1,Scientific Literature,Eur J Med Chem,2008.0,NaN,NaN,O=c1cc(-c2ccc(O)cc2)oc2cc(O)cc(O)c12,O=c1cc(-c2ccc(O)cc2)oc2cc(O)cc(O)c12,inactive
4,CHEMBL28,APIGENIN,NaN,270.24,0.0,2.58,12,O=c1cc(-c2ccc(O)cc2)oc2cc(O)cc(O)c12,IC50,'=',...,CHEMBL3621095,1,Scientific Literature,Eur J Med Chem,2015.0,NaN,NaN,O=c1cc(-c2ccc(O)cc2)oc2cc(O)cc(O)c12,O=c1cc(-c2ccc(O)cc2)oc2cc(O)cc(O)c12,inactive
5,CHEMBL28,APIGENIN,NaN,270.24,0.0,2.58,Apigenin,O=c1cc(-c2ccc(O)cc2)oc2cc(O)cc(O)c12,IC50,'=',...,CHEMBL1144205,1,Scientific Literature,Bioorg Med Chem,2008.0,NaN,NaN,O=c1cc(-c2ccc(O)cc2)oc2cc(O)cc(O)c12,O=c1cc(-c2ccc(O)cc2)oc2cc(O)cc(O)c12,active
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2479,CHEMBL224462,NaN,NaN,313.36,0.0,3.22,T46,CCC(c1ccc(C#N)cc1)(c1ccc(C#N)cc1)n1cncn1,IC50,'=',...,CHEMBL1139479,1,Scientific Literature,J Med Chem,2005.0,NaN,NaN,CCC(c1ccc(C#N)cc1)(c1ccc(C#N)cc1)n1cncn1,CCC(c1ccc(C#N)cc1)(c1ccc(C#N)cc1)n1cncn1,active
2480,CHEMBL4632445,NaN,NaN,358.40,0.0,3.98,3a,CCC#CCOc1ccc2c(=O)c3ccc(Cn4ccnc4)cc3oc2c1,IC50,'=',...,CHEMBL4610109,1,Scientific Literature,ACS Med Chem Lett,2020.0,NaN,NaN,CCC#CCOc1ccc2c(=O)c3ccc(Cn4ccnc4)cc3oc2c1,CCC#CCOc1ccc2c(=O)c3ccc(Cn4ccnc4)cc3oc2c1,active
2481,CHEMBL4634777,NaN,NaN,358.40,0.0,3.98,1a,CCC#CCOc1cc(Cn2ccnc2)c2oc3ccccc3c(=O)c2c1,IC50,'=',...,CHEMBL4610109,1,Scientific Literature,ACS Med Chem Lett,2020.0,NaN,NaN,CCC#CCOc1cc(Cn2ccnc2)c2oc3ccccc3c(=O)c2c1,CCC#CCOc1cc(Cn2ccnc2)c2oc3ccccc3c(=O)c2c1,inactive
2482,CHEMBL4755831,NaN,NaN,383.45,0.0,4.54,26,Nc1ccc(C(Cn2ccnc2)=C(c2ccc(O)cc2)c2ccc(O)cc2)cc1,IC50,'=',...,CHEMBL4715691,1,Scientific Literature,J Med Chem,2016.0,NaN,TIME = 0.5 hr,Nc1ccc(C(Cn2ccnc2)=C(c2ccc(O)cc2)c2ccc(O)cc2)cc1,Nc1ccc(C(=Cn2ccnc2)C(c2ccc(O)cc2)c2ccc(O)cc2)cc1,active


In [21]:
duplicates = preprocessed_df.duplicated('preprocessedSmiles', keep=False)

# Filter DataFrame to list duplicates with their information
duplicates_df = preprocessed_df[duplicates]
duplicates_df

,Molecule ChEMBL ID,Molecule Name,Molecule Max Phase,Molecular Weight,#RO5 Violations,AlogP,Compound Key,Smiles,Standard Type,Standard Relation,...,Document ChEMBL ID,Source ID,Source Description,Document Journal,Document Year,Cell ChEMBL ID,Properties,rawSmiles,preprocessedSmiles,activity_classification
0,CHEMBL174579,NaN,NaN,275.15,0.0,3.66,9b,Brc1ccc2c(c1)CC/C2=C/c1c[nH]cn1,IC50,'=',...,CHEMBL1144350,1,Scientific Literature,J Med Chem,2005.0,NaN,NaN,Brc1ccc2c(c1)CC/C2=C/c1c[nH]cn1,Brc1ccc2c(c1)CCC2=Cc1c[nH]cn1,active
2,CHEMBL28,APIGENIN,NaN,270.24,0.0,2.58,Apigenin,O=c1cc(-c2ccc(O)cc2)oc2cc(O)cc(O)c12,IC50,'=',...,CHEMBL1157768,1,Scientific Literature,Bioorg Med Chem Lett,2010.0,NaN,NaN,O=c1cc(-c2ccc(O)cc2)oc2cc(O)cc(O)c12,O=c1cc(-c2ccc(O)cc2)oc2cc(O)cc(O)c12,active
3,CHEMBL28,APIGENIN,NaN,270.24,0.0,2.58,35,O=c1cc(-c2ccc(O)cc2)oc2cc(O)cc(O)c12,IC50,'=',...,CHEMBL1156709,1,Scientific Literature,Eur J Med Chem,2008.0,NaN,NaN,O=c1cc(-c2ccc(O)cc2)oc2cc(O)cc(O)c12,O=c1cc(-c2ccc(O)cc2)oc2cc(O)cc(O)c12,inactive
4,CHEMBL28,APIGENIN,NaN,270.24,0.0,2.58,12,O=c1cc(-c2ccc(O)cc2)oc2cc(O)cc(O)c12,IC50,'=',...,CHEMBL3621095,1,Scientific Literature,Eur J Med Chem,2015.0,NaN,NaN,O=c1cc(-c2ccc(O)cc2)oc2cc(O)cc(O)c12,O=c1cc(-c2ccc(O)cc2)oc2cc(O)cc(O)c12,inactive
5,CHEMBL28,APIGENIN,NaN,270.24,0.0,2.58,Apigenin,O=c1cc(-c2ccc(O)cc2)oc2cc(O)cc(O)c12,IC50,'=',...,CHEMBL1144205,1,Scientific Literature,Bioorg Med Chem,2008.0,NaN,NaN,O=c1cc(-c2ccc(O)cc2)oc2cc(O)cc(O)c12,O=c1cc(-c2ccc(O)cc2)oc2cc(O)cc(O)c12,active
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2421,CHEMBL314351,NaN,NaN,260.34,0.0,2.29,Page 2677,CCCN1C(=O)CC[C@](CC)(c2ccncc2)C1=O,IC50,'=',...,CHEMBL1125058,1,Scientific Literature,J Med Chem,1990.0,NaN,NaN,CCCN1C(=O)CC[C@](CC)(c2ccncc2)C1=O,CCCN1C(=O)CCC(CC)(c2ccncc2)C1=O,inactive
2428,CHEMBL92488,NaN,NaN,251.29,0.0,3.00,5,O=C1/C(=C/c2ccncc2)CCc2c(O)cccc21,IC50,'=',...,CHEMBL1125863,1,Scientific Literature,J Med Chem,1991.0,NaN,NaN,O=C1/C(=C/c2ccncc2)CCc2c(O)cccc21,O=C1C(=Cc2ccncc2)CCc2c(O)cccc21,active
2429,CHEMBL92488,NaN,NaN,251.29,0.0,3.00,T55,O=C1/C(=C/c2ccncc2)CCc2c(O)cccc21,IC50,'=',...,CHEMBL1139479,1,Scientific Literature,J Med Chem,2005.0,NaN,NaN,O=C1/C(=C/c2ccncc2)CCc2c(O)cccc21,O=C1C(=Cc2ccncc2)CCc2c(O)cccc21,active
2460,CHEMBL132072,NaN,NaN,273.38,0.0,3.24,1,C[C@@H]1C(OC(=O)Cc2ccncc2)CC2CC1C2(C)C,IC50,'=',...,CHEMBL1128320,1,Scientific Literature,J Med Chem,1995.0,NaN,NaN,C[C@@H]1C(OC(=O)Cc2ccncc2)CC2CC1C2(C)C,CC1C(OC(=O)Cc2ccncc2)CC2CC1C2(C)C,active


In [22]:
groups = duplicates_df.groupby('preprocessedSmiles')
id_toremove = []
for name, group in groups:
    # Check if all values in 'activity_classification' are the same within the group
    if not group['activity_classification'].nunique() == 1:
        # Print out 'Molecule ChEMBL ID' for all entries in the group
        print(f"Different activity classifications found for {name}:")
        for chembl_id in group['Molecule ChEMBL ID']:
            print(chembl_id)
            id_toremove.append(chembl_id)

Different activity classifications found for C=C1CC2C3CCC(=O)C3(C)CCC2C2(C)C1=CC(=O)C1OC12:
CHEMBL3407537
CHEMBL3407537
Different activity classifications found for CC(=O)N1CCN(c2ccc(OCC3COC(Cn4ccnc4)(c4ccc(Cl)cc4Cl)O3)cc2)CC1:
CHEMBL157101
CHEMBL157101
CHEMBL157101
CHEMBL157101
CHEMBL157101
CHEMBL157101
CHEMBL157101
CHEMBL157101
CHEMBL157101
CHEMBL157101
CHEMBL157101
CHEMBL328863
CHEMBL75
CHEMBL75
CHEMBL75
CHEMBL75
CHEMBL319160
Different activity classifications found for CC(C)=CCc1cc(C2CC(=O)c3ccc(O)cc3O2)ccc1O:
CHEMBL558103
CHEMBL389924
CHEMBL389924
CHEMBL389924
Different activity classifications found for CC12CCC3C(CC(Cc4ccccc4)=C4CC(=O)CCC43C)C1CCC2=O:
CHEMBL24897
CHEMBL25523
Different activity classifications found for CC12CCC3C(CC(O)C4=CCCCC43CO)C1CCC2=O:
CHEMBL1630266
CHEMBL1630266
CHEMBL1630267
CHEMBL1630267
Different activity classifications found for CC12CCC3C(CC=C4C(O)CCCC43CO)C1CCC2=O:
CHEMBL1630273
CHEMBL1630273
CHEMBL1630274
CHEMBL1630274
Different activity classificatio

In [23]:
print(len(id_toremove))

170


In [24]:
print(len(set(id_toremove)))

49


In [25]:
# Drop the rows involved in the printed chembl_ids
for i in set(id_toremove):
    preprocessed_df = preprocessed_df[preprocessed_df['Molecule ChEMBL ID'] != i]

In [26]:
preprocessed_df

,Molecule ChEMBL ID,Molecule Name,Molecule Max Phase,Molecular Weight,#RO5 Violations,AlogP,Compound Key,Smiles,Standard Type,Standard Relation,...,Document ChEMBL ID,Source ID,Source Description,Document Journal,Document Year,Cell ChEMBL ID,Properties,rawSmiles,preprocessedSmiles,activity_classification
0,CHEMBL174579,NaN,NaN,275.15,0.0,3.66,9b,Brc1ccc2c(c1)CC/C2=C/c1c[nH]cn1,IC50,'=',...,CHEMBL1144350,1,Scientific Literature,J Med Chem,2005.0,NaN,NaN,Brc1ccc2c(c1)CC/C2=C/c1c[nH]cn1,Brc1ccc2c(c1)CCC2=Cc1c[nH]cn1,active
6,CHEMBL196328,NaN,NaN,314.18,0.0,4.67,BDBM8911,COc1ccc2cc(-c3cccnc3)ccc2c1Br,IC50,'=',...,CHEMBL3886626,37,BindingDB Database,NaN,2016.0,NaN,NaN,COc1ccc2cc(-c3cccnc3)ccc2c1Br,COc1ccc2cc(-c3cccnc3)ccc2c1Br,inactive
7,CHEMBL196328,NaN,NaN,314.18,0.0,4.67,10,COc1ccc2cc(-c3cccnc3)ccc2c1Br,IC50,'=',...,CHEMBL1141125,1,Scientific Literature,J Med Chem,2005.0,NaN,NaN,COc1ccc2cc(-c3cccnc3)ccc2c1Br,COc1ccc2cc(-c3cccnc3)ccc2c1Br,inactive
8,CHEMBL306022,NaN,NaN,302.50,1.0,5.34,15b,CC[C@@H]1CC2C(CCC3(C)C(O)CCC23)C2(C)CCCC=C12,IC50,'=',...,CHEMBL1129695,1,Scientific Literature,J Med Chem,1996.0,NaN,NaN,CC[C@@H]1CC2C(CCC3(C)C(O)CCC23)C2(C)CCCC=C12,CCC1CC2C(CCC3(C)C(O)CCC23)C2(C)CCCC=C12,active
9,CHEMBL495567,NaN,NaN,335.79,1.0,5.19,"9, NSC-12999",COc1ccc2nc3cc(Cl)ccc3c(Nc3cccnc3)c2c1,IC50,'=',...,CHEMBL1139349,1,Scientific Literature,J Med Chem,2009.0,NaN,NaN,COc1ccc2nc3cc(Cl)ccc3c(Nc3cccnc3)c2c1,COc1ccc2nc3cc(Cl)ccc3c(Nc3cccnc3)c2c1,active
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2479,CHEMBL224462,NaN,NaN,313.36,0.0,3.22,T46,CCC(c1ccc(C#N)cc1)(c1ccc(C#N)cc1)n1cncn1,IC50,'=',...,CHEMBL1139479,1,Scientific Literature,J Med Chem,2005.0,NaN,NaN,CCC(c1ccc(C#N)cc1)(c1ccc(C#N)cc1)n1cncn1,CCC(c1ccc(C#N)cc1)(c1ccc(C#N)cc1)n1cncn1,active
2480,CHEMBL4632445,NaN,NaN,358.40,0.0,3.98,3a,CCC#CCOc1ccc2c(=O)c3ccc(Cn4ccnc4)cc3oc2c1,IC50,'=',...,CHEMBL4610109,1,Scientific Literature,ACS Med Chem Lett,2020.0,NaN,NaN,CCC#CCOc1ccc2c(=O)c3ccc(Cn4ccnc4)cc3oc2c1,CCC#CCOc1ccc2c(=O)c3ccc(Cn4ccnc4)cc3oc2c1,active
2481,CHEMBL4634777,NaN,NaN,358.40,0.0,3.98,1a,CCC#CCOc1cc(Cn2ccnc2)c2oc3ccccc3c(=O)c2c1,IC50,'=',...,CHEMBL4610109,1,Scientific Literature,ACS Med Chem Lett,2020.0,NaN,NaN,CCC#CCOc1cc(Cn2ccnc2)c2oc3ccccc3c(=O)c2c1,CCC#CCOc1cc(Cn2ccnc2)c2oc3ccccc3c(=O)c2c1,inactive
2482,CHEMBL4755831,NaN,NaN,383.45,0.0,4.54,26,Nc1ccc(C(Cn2ccnc2)=C(c2ccc(O)cc2)c2ccc(O)cc2)cc1,IC50,'=',...,CHEMBL4715691,1,Scientific Literature,J Med Chem,2016.0,NaN,TIME = 0.5 hr,Nc1ccc(C(Cn2ccnc2)=C(c2ccc(O)cc2)c2ccc(O)cc2)cc1,Nc1ccc(C(=Cn2ccnc2)C(c2ccc(O)cc2)c2ccc(O)cc2)cc1,active


In [27]:
preprocessed_df.drop_duplicates('preprocessedSmiles',inplace=True)
preprocessed_df

,Molecule ChEMBL ID,Molecule Name,Molecule Max Phase,Molecular Weight,#RO5 Violations,AlogP,Compound Key,Smiles,Standard Type,Standard Relation,...,Document ChEMBL ID,Source ID,Source Description,Document Journal,Document Year,Cell ChEMBL ID,Properties,rawSmiles,preprocessedSmiles,activity_classification
0,CHEMBL174579,NaN,NaN,275.15,0.0,3.66,9b,Brc1ccc2c(c1)CC/C2=C/c1c[nH]cn1,IC50,'=',...,CHEMBL1144350,1,Scientific Literature,J Med Chem,2005.0,NaN,NaN,Brc1ccc2c(c1)CC/C2=C/c1c[nH]cn1,Brc1ccc2c(c1)CCC2=Cc1c[nH]cn1,active
6,CHEMBL196328,NaN,NaN,314.18,0.0,4.67,BDBM8911,COc1ccc2cc(-c3cccnc3)ccc2c1Br,IC50,'=',...,CHEMBL3886626,37,BindingDB Database,NaN,2016.0,NaN,NaN,COc1ccc2cc(-c3cccnc3)ccc2c1Br,COc1ccc2cc(-c3cccnc3)ccc2c1Br,inactive
8,CHEMBL306022,NaN,NaN,302.50,1.0,5.34,15b,CC[C@@H]1CC2C(CCC3(C)C(O)CCC23)C2(C)CCCC=C12,IC50,'=',...,CHEMBL1129695,1,Scientific Literature,J Med Chem,1996.0,NaN,NaN,CC[C@@H]1CC2C(CCC3(C)C(O)CCC23)C2(C)CCCC=C12,CCC1CC2C(CCC3(C)C(O)CCC23)C2(C)CCCC=C12,active
9,CHEMBL495567,NaN,NaN,335.79,1.0,5.19,"9, NSC-12999",COc1ccc2nc3cc(Cl)ccc3c(Nc3cccnc3)c2c1,IC50,'=',...,CHEMBL1139349,1,Scientific Literature,J Med Chem,2009.0,NaN,NaN,COc1ccc2nc3cc(Cl)ccc3c(Nc3cccnc3)c2c1,COc1ccc2nc3cc(Cl)ccc3c(Nc3cccnc3)c2c1,active
10,CHEMBL495777,NaN,NaN,310.31,0.0,3.45,"11, NSC-131736",COc1ccc2c(Cc3cccc([N+](=O)[O-])c3)cncc2c1O,IC50,'=',...,CHEMBL1139349,1,Scientific Literature,J Med Chem,2009.0,NaN,NaN,COc1ccc2c(Cc3cccc([N+](=O)[O-])c3)cncc2c1O,COc1ccc2c(Cc3cccc([N+](=O)[O-])c3)cncc2c1O,inactive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2479,CHEMBL224462,NaN,NaN,313.36,0.0,3.22,T46,CCC(c1ccc(C#N)cc1)(c1ccc(C#N)cc1)n1cncn1,IC50,'=',...,CHEMBL1139479,1,Scientific Literature,J Med Chem,2005.0,NaN,NaN,CCC(c1ccc(C#N)cc1)(c1ccc(C#N)cc1)n1cncn1,CCC(c1ccc(C#N)cc1)(c1ccc(C#N)cc1)n1cncn1,active
2480,CHEMBL4632445,NaN,NaN,358.40,0.0,3.98,3a,CCC#CCOc1ccc2c(=O)c3ccc(Cn4ccnc4)cc3oc2c1,IC50,'=',...,CHEMBL4610109,1,Scientific Literature,ACS Med Chem Lett,2020.0,NaN,NaN,CCC#CCOc1ccc2c(=O)c3ccc(Cn4ccnc4)cc3oc2c1,CCC#CCOc1ccc2c(=O)c3ccc(Cn4ccnc4)cc3oc2c1,active
2481,CHEMBL4634777,NaN,NaN,358.40,0.0,3.98,1a,CCC#CCOc1cc(Cn2ccnc2)c2oc3ccccc3c(=O)c2c1,IC50,'=',...,CHEMBL4610109,1,Scientific Literature,ACS Med Chem Lett,2020.0,NaN,NaN,CCC#CCOc1cc(Cn2ccnc2)c2oc3ccccc3c(=O)c2c1,CCC#CCOc1cc(Cn2ccnc2)c2oc3ccccc3c(=O)c2c1,inactive
2482,CHEMBL4755831,NaN,NaN,383.45,0.0,4.54,26,Nc1ccc(C(Cn2ccnc2)=C(c2ccc(O)cc2)c2ccc(O)cc2)cc1,IC50,'=',...,CHEMBL4715691,1,Scientific Literature,J Med Chem,2016.0,NaN,TIME = 0.5 hr,Nc1ccc(C(Cn2ccnc2)=C(c2ccc(O)cc2)c2ccc(O)cc2)cc1,Nc1ccc(C(=Cn2ccnc2)C(c2ccc(O)cc2)c2ccc(O)cc2)cc1,active


In [28]:
# Check for duplicates in 'preprocessedSmiles'
duplicates = preprocessed_df.duplicated(subset='preprocessedSmiles', keep=False)

# Show rows where 'preprocessedSmiles' is duplicated
duplicate_rows = preprocessed_df[duplicates]
duplicate_rows

,Molecule ChEMBL ID,Molecule Name,Molecule Max Phase,Molecular Weight,#RO5 Violations,AlogP,Compound Key,Smiles,Standard Type,Standard Relation,...,Document ChEMBL ID,Source ID,Source Description,Document Journal,Document Year,Cell ChEMBL ID,Properties,rawSmiles,preprocessedSmiles,activity_classification


In [29]:
len(preprocessed_df[preprocessed_df['pChEMBL Value']>=6]),len(preprocessed_df[preprocessed_df['pChEMBL Value']<6])

(759, 626)

In [30]:
preprocessed_df_useful = preprocessed_df[['preprocessedSmiles','Molecule ChEMBL ID', 'activity_classification']]
preprocessed_df_useful

,preprocessedSmiles,Molecule ChEMBL ID,activity_classification
0,Brc1ccc2c(c1)CCC2=Cc1c[nH]cn1,CHEMBL174579,active
6,COc1ccc2cc(-c3cccnc3)ccc2c1Br,CHEMBL196328,inactive
8,CCC1CC2C(CCC3(C)C(O)CCC23)C2(C)CCCC=C12,CHEMBL306022,active
9,COc1ccc2nc3cc(Cl)ccc3c(Nc3cccnc3)c2c1,CHEMBL495567,active
10,COc1ccc2c(Cc3cccc([N+](=O)[O-])c3)cncc2c1O,CHEMBL495777,inactive
...,...,...,...
2479,CCC(c1ccc(C#N)cc1)(c1ccc(C#N)cc1)n1cncn1,CHEMBL224462,active
2480,CCC#CCOc1ccc2c(=O)c3ccc(Cn4ccnc4)cc3oc2c1,CHEMBL4632445,active
2481,CCC#CCOc1cc(Cn2ccnc2)c2oc3ccccc3c(=O)c2c1,CHEMBL4634777,inactive
2482,Nc1ccc(C(=Cn2ccnc2)C(c2ccc(O)cc2)c2ccc(O)cc2)cc1,CHEMBL4755831,active


In [31]:
actives = preprocessed_df_useful[preprocessed_df_useful['activity_classification'] == 'active']
inactives = preprocessed_df_useful[preprocessed_df_useful['activity_classification'] == 'inactive']

In [32]:
actives

,preprocessedSmiles,Molecule ChEMBL ID,activity_classification
0,Brc1ccc2c(c1)CCC2=Cc1c[nH]cn1,CHEMBL174579,active
8,CCC1CC2C(CCC3(C)C(O)CCC23)C2(C)CCCC=C12,CHEMBL306022,active
9,COc1ccc2nc3cc(Cl)ccc3c(Nc3cccnc3)c2c1,CHEMBL495567,active
15,O=[N+]([O-])c1ccc(C(c2cc3ccc(O)cc3o2)n2cncn2)cc1,CHEMBL202231,active
16,COc1ccc2cc(C(O)(c3ccc(Cl)cc3)c3cccnc3)oc2c1,CHEMBL202612,active
...,...,...,...
2476,CC(=C(c1ccc(O)cc1)c1ccc(O)cc1)c1c[nH]cn1,CHEMBL5082681,active
2479,CCC(c1ccc(C#N)cc1)(c1ccc(C#N)cc1)n1cncn1,CHEMBL224462,active
2480,CCC#CCOc1ccc2c(=O)c3ccc(Cn4ccnc4)cc3oc2c1,CHEMBL4632445,active
2482,Nc1ccc(C(=Cn2ccnc2)C(c2ccc(O)cc2)c2ccc(O)cc2)cc1,CHEMBL4755831,active


In [33]:
inactives

,preprocessedSmiles,Molecule ChEMBL ID,activity_classification
6,COc1ccc2cc(-c3cccnc3)ccc2c1Br,CHEMBL196328,inactive
10,COc1ccc2c(Cc3cccc([N+](=O)[O-])c3)cncc2c1O,CHEMBL495777,inactive
12,COc1ccc(C(=NNc2cnnc(Cl)c2Cl)c2ccccc2)cc1,CHEMBL497614,inactive
13,COc1ccc2c(=O)c(-c3ccccc3)c(-n3cncn3)oc2c1,CHEMBL190099,inactive
18,COc1ccc2c(=O)c(-c3ccccc3)c(SCc3cccnc3)oc2c1,CHEMBL185819,inactive
...,...,...,...
2468,CC(CCc1cccnc1)=C(c1ccc(O)cc1)c1ccc(O)cc1,CHEMBL5092843,inactive
2471,CC12CCC3C4=C(CCCC4)CC(=O)C3C1CCC2=O,CHEMBL71082,inactive
2474,COCOc1ccc2c(c1)OC(c1ccc(OCOC)c(CC=C(C)C)c1)CC2=O,CHEMBL229945,inactive
2475,COc1ccc2c(c1)OC(c1ccc(O)c(CC=C(C)C)c1)CC2=O,CHEMBL388374,inactive


In [34]:
#save the preprocessed dataframe active and inactives
actives.to_csv('./datasets/2_preprocessed/chembl_preprocessed_active.csv', index=False)
inactives.to_csv('./datasets/2_preprocessed/chembl_preprocessed_inactive.csv', index=False)

In [35]:
from rdkit.Chem import Descriptors

PandasTools.AddMoleculeColumnToFrame(preprocessed_df_useful,'preprocessedSmiles','Molecule')


/home/sliu/miniconda3/envs/cypscreen/lib/python3.8/site-packages/rdkit/Chem/PandasTools.py:425: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame[molCol] = frame[smilesCol].map(Chem.MolFromSmiles)


In [36]:
descriptors_list = [x[0] for x in Descriptors._descList]
print(descriptors_list)
print(len(descriptors_list))

['MaxEStateIndex', 'MinEStateIndex', 'MaxAbsEStateIndex', 'MinAbsEStateIndex', 'qed', 'MolWt', 'HeavyAtomMolWt', 'ExactMolWt', 'NumValenceElectrons', 'NumRadicalElectrons', 'MaxPartialCharge', 'MinPartialCharge', 'MaxAbsPartialCharge', 'MinAbsPartialCharge', 'FpDensityMorgan1', 'FpDensityMorgan2', 'FpDensityMorgan3', 'BCUT2D_MWHI', 'BCUT2D_MWLOW', 'BCUT2D_CHGHI', 'BCUT2D_CHGLO', 'BCUT2D_LOGPHI', 'BCUT2D_LOGPLOW', 'BCUT2D_MRHI', 'BCUT2D_MRLOW', 'BalabanJ', 'BertzCT', 'Chi0', 'Chi0n', 'Chi0v', 'Chi1', 'Chi1n', 'Chi1v', 'Chi2n', 'Chi2v', 'Chi3n', 'Chi3v', 'Chi4n', 'Chi4v', 'HallKierAlpha', 'Ipc', 'Kappa1', 'Kappa2', 'Kappa3', 'LabuteASA', 'PEOE_VSA1', 'PEOE_VSA10', 'PEOE_VSA11', 'PEOE_VSA12', 'PEOE_VSA13', 'PEOE_VSA14', 'PEOE_VSA2', 'PEOE_VSA3', 'PEOE_VSA4', 'PEOE_VSA5', 'PEOE_VSA6', 'PEOE_VSA7', 'PEOE_VSA8', 'PEOE_VSA9', 'SMR_VSA1', 'SMR_VSA10', 'SMR_VSA2', 'SMR_VSA3', 'SMR_VSA4', 'SMR_VSA5', 'SMR_VSA6', 'SMR_VSA7', 'SMR_VSA8', 'SMR_VSA9', 'SlogP_VSA1', 'SlogP_VSA10', 'SlogP_VSA11', 'Slo

In [37]:
print(np.__version__)


1.20.3


In [38]:
from rdkit.ML.Descriptors import MoleculeDescriptors
def compute_2Drdkit(mol):
    calc = MoleculeDescriptors.MolecularDescriptorCalculator([x[0] for x in Descriptors._descList])
    ds = calc.CalcDescriptors(mol)
    return list(ds)

In [39]:
preprocessed_df_useful['descriptor'] = preprocessed_df_useful.Molecule.apply(compute_2Drdkit)

/tmp/ipykernel_3406031/3671742272.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  preprocessed_df_useful['descriptor'] = preprocessed_df_useful.Molecule.apply(compute_2Drdkit)


In [40]:
#one descriptor per column
calc = MoleculeDescriptors.MolecularDescriptorCalculator([x[0] for x in Descriptors._descList])
desc_cols = list(calc.GetDescriptorNames())
preprocessed_df_useful[desc_cols] = preprocessed_df_useful.descriptor.to_list()

/tmp/ipykernel_3406031/2296068614.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  preprocessed_df_useful[desc_cols] = preprocessed_df_useful.descriptor.to_list()
/tmp/ipykernel_3406031/2296068614.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  preprocessed_df_useful[desc_cols] = preprocessed_df_useful.descriptor.to_list()
/tmp/ipykernel_3406031/2296068614.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

In [41]:
preprocessed_df_useful[desc_cols]

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,4.228995,1.005341,4.228995,1.005341,0.844932,275.149,264.061,274.010560,80.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,5.311073,0.848729,5.311073,0.848729,0.685407,314.182,302.086,313.010226,94.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,10.582239,-0.032752,10.582239,0.032752,0.648494,302.502,268.230,302.260966,124.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,6.139654,0.662212,6.139654,0.662212,0.523359,335.794,321.682,335.082540,118.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,10.879415,-0.415136,10.879415,0.040178,0.588994,310.309,296.197,310.095357,116.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2479,9.038998,-0.543183,9.038998,0.543183,0.740537,313.364,298.244,313.132745,116.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2480,12.788407,-0.043745,12.788407,0.043745,0.408911,358.397,340.253,358.131742,134.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2481,13.008467,-0.065904,13.008467,0.065904,0.408911,358.397,340.253,358.131742,134.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2482,9.765064,-0.136829,9.765064,0.136829,0.435979,383.451,362.283,383.163377,144.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
preprocessed_df_useful[desc_cols].describe()

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
count,1385.000000,1385.000000,1385.000000,1385.000000,1385.000000,1385.000000,1385.000000,1385.000000,1385.000000,1385.0,...,1385.000000,1385.000000,1385.000000,1385.000000,1385.000000,1385.000000,1385.0,1385.000000,1385.000000,1385.000000
mean,11.071659,-0.903910,11.071659,0.185405,0.590354,355.999362,335.986377,355.605424,131.070036,0.0,...,0.034657,0.177617,0.002888,0.005054,0.003610,0.026715,0.0,0.009386,0.150181,0.000722
std,2.640153,1.607475,2.640153,0.217916,0.176838,98.727569,95.686412,98.551761,34.026535,0.0,...,0.190710,0.465816,0.053683,0.070938,0.059997,0.165726,0.0,0.096462,0.769150,0.026870
min,2.402778,-5.325792,2.402778,0.000110,0.048436,250.301000,228.162000,250.110613,78.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
25%,9.765064,-0.794248,9.765064,0.043301,0.486521,298.386000,276.210000,298.168128,112.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
50%,12.215787,-0.239970,12.215787,0.100530,0.611633,333.391000,313.663000,333.147727,124.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
75%,12.785178,0.028280,12.785178,0.239697,0.723736,378.603000,360.283000,378.168722,140.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
max,16.050416,1.181950,16.050416,1.181950,0.933823,892.866000,859.602000,892.179952,320.000000,0.0,...,2.000000,2.000000,1.000000,1.000000,1.000000,2.000000,0.0,1.000000,10.000000,1.000000


In [43]:
preprocessed_df_useful[desc_cols].dropna(inplace=True)

/tmp/ipykernel_3406031/2119641864.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  preprocessed_df_useful[desc_cols].dropna(inplace=True)


In [44]:
preprocessed_df_useful[desc_cols].isnull().values.any()

False

In [45]:
preprocessed_df_useful.reset_index(inplace=True)

/tmp/ipykernel_3406031/1341088621.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  preprocessed_df_useful.reset_index(inplace=True)


In [46]:
#Morgan2 FP
def get_morgan2(mol):
    return list(AllChem.GetMorganFingerprintAsBitVect(mol,2,nBits=2048))

In [47]:
preprocessed_df_useful['fp'] = preprocessed_df_useful.Molecule.apply(get_morgan2) #doesn't work if save the fp in one column

/tmp/ipykernel_3406031/640373380.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  preprocessed_df_useful['fp'] = preprocessed_df_useful.Molecule.apply(get_morgan2) #doesn't work if save the fp in one column


In [48]:
morgan2_cols = ['morgan2_b'+str(i) for i in list(range(2048))]
preprocessed_df_useful[morgan2_cols] = preprocessed_df_useful.fp.to_list()

/tmp/ipykernel_3406031/538811567.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  preprocessed_df_useful[morgan2_cols] = preprocessed_df_useful.fp.to_list()
/tmp/ipykernel_3406031/538811567.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  preprocessed_df_useful[morgan2_cols] = preprocessed_df_useful.fp.to_list()
/tmp/ipykernel_3406031/538811567.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns

In [49]:
#print scikit-learn version
import sklearn
print(sklearn.__version__)

1.3.2


In [50]:
#shuffel the data when train and test split
from sklearn.model_selection import train_test_split
subset1, subset2 = train_test_split(preprocessed_df_useful, test_size= 0.2, random_state=42)
print(len(subset1), len(subset2))

1108 277


In [51]:
subset1["activity_classification"].value_counts(), subset2["activity_classification"].value_counts()

(activity_classification
 active      610
 inactive    498
 Name: count, dtype: int64,
 activity_classification
 active      149
 inactive    128
 Name: count, dtype: int64)

In [52]:
subset1.to_csv('./datasets/3_train_test_split/chembl_trainset_featurized.csv', index=False)
subset2.to_csv('./datasets/3_train_test_split/chembl_testset_featurized.csv', index=False)

In [53]:
subset1_withoutfeatures = subset1[["preprocessedSmiles", "Molecule ChEMBL ID", "activity_classification"]]
subset2_withoutfeatures = subset2[["preprocessedSmiles", "Molecule ChEMBL ID", "activity_classification"]]
subset1_withoutfeatures.to_csv('./datasets/3_train_test_split/chembl_trainset.csv', index=False)
subset2_withoutfeatures.to_csv('./datasets/3_train_test_split/chembl_testset.csv', index=False)